# Data Cleaning 

#### 1. Import pandas library.

In [1]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data. 


In [4]:
from sqlalchemy import create_engine

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/dataset/Stats).

In [ ]:
driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'sakila'

In [ ]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'

In [ ]:
engine = create_engine(connection_string)

#### 4. Import the users table.

In [ ]:
query = """
        SELECT * FROM users
"""

In [ ]:
users = pd.read_sql(query, engine)

In [ ]:
users.head()

#### 5. Rename Id column to userId.

In [ ]:
users = users.rename(columns={"Id column":"userId"})

#### 6. Import the posts table. 

In [7]:
query = """
        SELECT * FROM posts
"""

In [ ]:
posts = pd.read_sql(query, engine)

In [ ]:
posts.head()

#### 7. Rename Id column to postId and OwnerUserId to userId.

In [8]:
posts = posts.rename(columns={"OwnerUserId":"userId"})

#### 8. Define new dataframes for users and posts with the following selected columns:
**users columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts columns**: postId, Score, userID, ViewCount, CommentCount

In [ ]:
users.columns

In [ ]:
users = users.rename(columns={"XXXX":"userId", "XXXX":"Reputation", "XXXX":"Views", "XXXX":"UpVotes", "XXXX":"DownVotes"})

In [ ]:
posts.columns

In [ ]:
posts = posts.rename(columns={"XXXX":"postId", "XXXX":"Score", "XXXX":"userID", "XXXX":"ViewCount", "XXXX":"CommentCount"})

#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [11]:
df_merge = users.merge(posts, left_on='userId', right_on='userID')

#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [ ]:
# Finding missing values:
# https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b 
    

#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 